## Retrieval ChatBot
QQ匹配，question和question进行匹配
（1）先用dual_model选topk
（2）再用cross_model选最终答案

In [1]:
! pip install faiss-cpu

Looking in indexes: https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 4.3 MB/s eta 0:00:0000:0100:01


### Step1 读取faq数据

In [1]:
import pandas as pd

data = pd.read_csv("law_faq.csv")
data.head()

,title,reply
0,在法律中定金与订金的区别订金和定金哪个受,“定金”是指当事人约定由一方向对方给付的，作为债权担保的一定数额的货币，它属于一种法律上的担...
1,盗窃罪的犯罪客体是什么，盗窃罪的犯罪主体,盗窃罪的客体要件本罪侵犯的客体是公私财物的所有权。侵犯的对象，是国家、集体或个人的财物，一般...
2,非法微整形机构构成非法经营罪吗,符合要件就有可能。非法经营罪，是指未经许可经营专营、专卖物品或其他限制买卖的物品，买卖进出口...
3,入室持刀行凶伤人能不能判刑,对于入室持刀伤人涉嫌故意伤害刑事犯罪，一经定罪，故意伤害他人身体的，处三年以下有期徒刑、拘役...
4,对交通事故责任认定书不服怎么办，交通事故损,事故认定书下发后，如果你对认定不满意，可在接到认定书3日内到上一级公安机关复议。


### Step2 加载模型

In [2]:
from transformers import BertPreTrainedModel, BertModel, BertConfig
from typing import Optional
from transformers.configuration_utils import PretrainedConfig
from torch.nn import CosineSimilarity, CosineEmbeddingLoss
import torch

class DualModel(BertPreTrainedModel):
    def __init__(self, config, dim=0, *inputs, **kwargs):
        super().__init__(config, *inputs, **kwargs)
        self.bert = BertModel(config)
        self.dim = dim
        self.cosine_similarity = CosineSimilarity(dim=dim)  # 初始化时指定dim
        # Initialize weights and apply final processing
        self.post_init()

    @classmethod
    def from_pretrained(cls, config_name, dim=1, *inputs, **kwargs):
        config = BertConfig.from_pretrained(config_name)
        model = cls(config, dim, *inputs, **kwargs)
        model.bert = BertModel.from_pretrained(config_name)
        return model
    
    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        # Step1 分别获取 sen1 和 sen2 的输入
        sen1_input_ids, sen2_input_ids = input_ids[:, 0], input_ids[:, 1]
        sen1_attention_mask, sen2_attention_mask = attention_mask[:, 0], attention_mask[:, 1]
        sen1_token_type_ids, sen2_token_type_ids = token_type_ids[:, 0], token_type_ids[:, 1]

        # Step2 分别获取 sen1 和 sen2 的向量表示
        sen1_outputs = self.bert(
            sen1_input_ids,
            attention_mask=sen1_attention_mask,
            token_type_ids=sen1_token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        sen1_pooled_output = sen1_outputs[1]  # [batch, hidden(768)]

        sen2_outputs = self.bert(
            sen2_input_ids,
            attention_mask=sen2_attention_mask,
            token_type_ids=sen2_token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        sen2_pooled_output = sen2_outputs[1]  # [batch, hidden(768)]

        # Step3 计算相似度
        cos = self.cosine_similarity(sen1_pooled_output, sen2_pooled_output)  # [batch,]

        # Step4 计算loss
        loss = None
        if labels is not None:
            loss_fct = CosineEmbeddingLoss(margin=0.3)
            loss = loss_fct(sen1_pooled_output, sen2_pooled_output, labels)
        
        output = (cos,)
        return ((loss,) + output) if loss is not None else output

In [16]:
dual_model = DualModel.from_pretrained("dual_model/checkpoint-375/")
dual_model = dual_model.cuda()
dual_model.eval()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DualModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")

### Step3 将数据集问题编码成向量

In [4]:
questions = data["title"]
questions

0                                在法律中定金与订金的区别订金和定金哪个受
1                                盗窃罪的犯罪客体是什么，盗窃罪的犯罪主体
2                                     非法微整形机构构成非法经营罪吗
3                                       入室持刀行凶伤人能不能判刑
4                               对交通事故责任认定书不服怎么办，交通事故损
                             ...                     
18208     在一个APP贷款，3500元，18期还款，算下来需要换6000元，请问贷款方是否违法？
18209                               没交社保，只有工资，向哪个部门投诉
18210                                 老赖欠钱不还，可以用坐牢抵债吗
18211    我男朋友把我姐夫的车给偷了，我姐夫报案了，一开始我不知道他偷车了我知道后帮他逃跑我有罪吗
18212                                     个人与个人纠纷怎么处理
Name: title, Length: 18213, dtype: object

In [14]:
questions = data["title"].to_list()
questions

['在法律中定金与订金的区别订金和定金哪个受',
 '盗窃罪的犯罪客体是什么，盗窃罪的犯罪主体',
 '非法微整形机构构成非法经营罪吗',
 '入室持刀行凶伤人能不能判刑',
 '对交通事故责任认定书不服怎么办，交通事故损',
 '喝完就又去唱歌回家时去了车祸去唱歌我同意了车是我开的',
 '二审一般要多久',
 '欠钱没有欠条有录音能起诉吗',
 '从合伙经营公司撤资，只给欠条有用吗',
 '哪些情况可以先行刑事拘留',
 '是一个撞了人的不陪偿',
 '在工地上面出了安全事故怎么来维护自己的权',
 '申请和受理劳动争议调解仲裁时间上有什么限制',
 '商标注册有什么相对禁止的条件',
 '高利贷还不还不知如何是好。',
 '公司破产孕妇怎么赔偿',
 '两个人打架,因为对方先打我,后来找着打他的最后对方叫来五六人把我的肋骨打断',
 '客户拖欠货款',
 '到法院起诉一个人，要多少费用',
 '签了购房合同等于取得房屋产权吗',
 '迷信诈骗团伙老板和骨干会被判刑多少年',
 '婚后买房，婆婆出了一半钱，离婚房怎么分',
 '刑事案件，怎么收费，请律师都是有哪些费用',
 '我是个大车司机，老板拖欠工资我用什么方法能',
 '交通事故对方主责我次责车辆修理费怎么算',
 '你好，花了别人偷来送给你的钱，在公安局画押教罚款会不会案底',
 '请问，父亲生前欠下债务要还吗',
 '起诉欠款人的可行法律流程',
 '公司拖欠工资打什么电话投诉才可以',
 '结婚二+多年没在一起算是自动离婚不',
 '钱站利息算高利贷吗？',
 '你好，在公安局的笔录上画了押，还教了罚款会不会有案底',
 '假如房产证和户口本同时给租赁人办理营业执照会发生什么后果？',
 '请问我是二婚，房子是婚前买的，上午办的结婚证，下午去加的老婆名字，这样算不算夫妻共同财产',
 '是轻伤，他现在被拘留了还是拘留几天就放了',
 '如何区分贪污和受贿？',
 '公司法人变更工程代建合同是否还有效',
 '带未满周岁孩子无经济来源的签订担保有效吗',
 '女方欠男方的钱，离婚协议书要不要写让她归还？',
 '购房合同丢了怎么办',
 '新房与效果图不一致可否退房',
 '公司有权没收手机吗？',
 '通*亚*拖*工*工*',
 '老板拖欠工资拒绝给押金怎么办',
 '您好，我现在是监外执行期间，信

In [17]:
import torch
from tqdm import tqdm

vectors = []
with torch.inference_mode():
    for i in tqdm(range(0, len(questions), 32)):
        batch_sens = questions[i: i + 32]
        # print(len(batch_sens))
        # print(type(batch_sens), batch_sens)
        inputs = tokenizer(batch_sens, return_tensors="pt", padding=True, max_length=128, truncation=True)
        inputs = {k : v.to(dual_model.device) for k, v in inputs.items()}
        vector = dual_model.bert(**inputs)[1]
        vectors.append(vector)
vectors = torch.concat(vectors, dim=0).cpu().numpy()
vectors.shape

100%|██████████| 570/570 [00:10<00:00, 53.94it/s]


(18213, 768)

### Step4 创建索引

In [19]:
import faiss

index = faiss.IndexFlatIP(768)
faiss.normalize_L2(vectors)
index.add(vectors)
index

<faiss.swigfaiss_avx512.IndexFlatIP; proxy of <Swig Object of type 'faiss::IndexFlatIP *' at 0x7fdf3378e4e0> >

### Step5 对用户问题进行向量编码

In [49]:
question = "寻衅滋事"
with torch.inference_mode():
    inputs = tokenizer(question, return_tensors="pt", padding=True, max_length=128, truncation=True)
    inputs = {k : v.to(dual_model.device) for k, v in inputs.items()}
    q_vector = dual_model.bert(**inputs)[1]
    q_vector = q_vector.cpu().numpy()
q_vector.shape

(1, 768)

### Step6 向量匹配(召回)

In [62]:
faiss.normalize_L2(vector)
topk = 10
scores, indexes = index.search(q_vector, topk)
print(scores, indexes)
match_questions = [None] * topk
final_answers = [None] * topk
for i in range(0, topk):
    match_questions[i] = questions[indexes[0][i]]
    final_answers[i] = data["reply"].tolist()[indexes[0][i]]

print(match_questions)
print(final_answers)

[[18.840733 18.378693 17.732174 17.586985 17.586533 17.464132 17.32861
  17.325531 17.244131 17.239017]] [[  539  7292   230  2269 17626  8777 10390  9819 13298  8625]]
['涉嫌寻衅滋事', '两个轻微伤够寻衅滋事', '轻伤一级不赔钱判几年', '农民工讨薪', '元*县*管拉幅讨薪', '建筑工伤不解决就不出院', '诈骗30万全部退还', '交通事故肇事逃逸', '黑*江*集*长*煤*拖欠工资', '醉驾请假']
['说明具有寻衅滋事行为，应受到相应的处罚，行为人情形严重或行为恶劣的涉嫌了寻衅滋事罪。寻衅滋事是指行为人结伙斗殴的、追逐、拦截他人的、强拿硬要或者任意损毁、占用公私财物的、其他寻衅滋事的行为。寻衅滋事罪，是指在公共场所无事生非、起哄闹事，造成公共场所秩序严重混乱的，追逐、拦截、辱骂、恐吓他人，强拿硬要或者任意损毁、占用公私财物，破坏社会秩序，情节严重的行为。对于寻衅滋事行为的处罚：1、《中华人*共和国治安管理处罚法》第二十六条规定，有下列行为之一的，处五日以上十日以下拘留，可以并处五百元以下罚款;情节较重的，处十日以上十五日以下拘留，可以并处一千元以下罚款:(一)结伙斗殴的;(二)追逐、拦截他人的;(三)强拿硬要或者任意损毁、占用公私财物的;(四)其他寻衅滋事行为;2、《中华人*共和国刑法》第二百九十三条有下列寻衅滋事行为之一，破坏社会秩序的，处五年以下有期徒刑、拘役或者管制:(一)随意殴打他人，情节恶劣的;(二)追逐、拦截、辱骂、恐吓他人，情节恶劣的;(三)强拿硬要或者任意损毁、占用公私财物，情节严重的;(四)在公共场所起哄闹事。造成公共场所秩序严重混乱的。纠集他人多次实施前款行为，严重破坏社会秩序的，处五年以上十年以下有期徒刑，可以并处罚金。3、最*人*法*和最*人*检**《关于办理寻衅滋事案件的司法解释》为依法惩治寻衅滋事犯罪，维护社会秩序，最*人*法*会*最*人*检**根据《中华人*共和国刑法》的有关规定，就办理寻衅滋事刑事案件适用法律的若干问题司法解释如下:第一条行为人为寻求刺激、发泄情绪、逞强耍横等，无事生非，实施刑法第二百九十三条规定的行为的，应当认定为"寻衅滋事"。行

### Step7 加载交互模型

In [51]:
from transformers import BertForSequenceClassification
cross_model = BertForSequenceClassification.from_pretrained("cross_model/checkpoint-750/")
cross_model = cross_model.cuda()
cross_model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

### Step8 最终预测(排序)

In [52]:
question = "寻衅滋事"
question = [question] * len(match_questions)
print(question)
# question
inputs = tokenizer(question, match_questions, return_tensors="pt", padding=True, max_length=128, truncation=True)
inputs = {k: v.to(cross_model.device) for k, v in inputs.items()}
with torch.inference_mode():
    logits = cross_model(**inputs).logits.squeeze()
    result = torch.argmax(logits, dim=-1)
print(logits, result)

['寻衅滋事', '寻衅滋事', '寻衅滋事', '寻衅滋事', '寻衅滋事', '寻衅滋事', '寻衅滋事', '寻衅滋事', '寻衅滋事', '寻衅滋事']
tensor([ 0.9684,  0.3726, -0.0722,  0.3085,  0.0283,  0.1325, -0.0924,  0.0717,
        -0.1414,  0.0640], device='cuda:0') tensor(0, device='cuda:0')


In [64]:
match_question = match_questions[result.item()]
final_answers[i] = final_answers[result.item()]
print(match_question)
print(final_answers)

涉嫌寻衅滋事
['说明具有寻衅滋事行为，应受到相应的处罚，行为人情形严重或行为恶劣的涉嫌了寻衅滋事罪。寻衅滋事是指行为人结伙斗殴的、追逐、拦截他人的、强拿硬要或者任意损毁、占用公私财物的、其他寻衅滋事的行为。寻衅滋事罪，是指在公共场所无事生非、起哄闹事，造成公共场所秩序严重混乱的，追逐、拦截、辱骂、恐吓他人，强拿硬要或者任意损毁、占用公私财物，破坏社会秩序，情节严重的行为。对于寻衅滋事行为的处罚：1、《中华人*共和国治安管理处罚法》第二十六条规定，有下列行为之一的，处五日以上十日以下拘留，可以并处五百元以下罚款;情节较重的，处十日以上十五日以下拘留，可以并处一千元以下罚款:(一)结伙斗殴的;(二)追逐、拦截他人的;(三)强拿硬要或者任意损毁、占用公私财物的;(四)其他寻衅滋事行为;2、《中华人*共和国刑法》第二百九十三条有下列寻衅滋事行为之一，破坏社会秩序的，处五年以下有期徒刑、拘役或者管制:(一)随意殴打他人，情节恶劣的;(二)追逐、拦截、辱骂、恐吓他人，情节恶劣的;(三)强拿硬要或者任意损毁、占用公私财物，情节严重的;(四)在公共场所起哄闹事。造成公共场所秩序严重混乱的。纠集他人多次实施前款行为，严重破坏社会秩序的，处五年以上十年以下有期徒刑，可以并处罚金。3、最*人*法*和最*人*检**《关于办理寻衅滋事案件的司法解释》为依法惩治寻衅滋事犯罪，维护社会秩序，最*人*法*会*最*人*检**根据《中华人*共和国刑法》的有关规定，就办理寻衅滋事刑事案件适用法律的若干问题司法解释如下:第一条行为人为寻求刺激、发泄情绪、逞强耍横等，无事生非，实施刑法第二百九十三条规定的行为的，应当认定为"寻衅滋事"。行为人因日常生活中的偶发矛盾纠纷，借故生非，实施刑法第二百九十三条规定的行为的，应当认定为"寻衅滋事"，但矛盾系由被害人故意引发或者被害人对矛盾激化负有主要责任的除外。行为人因婚恋、家庭、邻里、债务等纠纷，实施殴打、辱骂、恐吓他人或者损毁、占用他人财物等行为的，一般不认定为"寻衅滋事"，但经有关部门批评制止或者处理处罚后，继续实施前列行为，破坏社会秩序的除外。第二条随意殴打他人，破坏社会秩序，具有下列情形之一的，应当认定为刑法第二百九十三条第一款第一项规定的"情节恶劣":1、致一人以上轻伤或者二人以上轻微伤的;2、引起他人精神失常、自杀等严重后果的;3、多次随意殴打他人的;